In [1]:
import os
import requests
import html
from datetime import datetime
from dotenv import load_dotenv
from openai import AzureOpenAI
from IPython.display import HTML, Markdown, display
from src import aoai_helpers as helpers
from src import aoai_model_configs as model_configs

load_dotenv()  

apim_key = os.environ['APIM_KEY']  
apim_endpoint = os.environ['APIM_ENDPOINT']  
version_of_api = os.environ['AOAI_API_VERSION']
bing_subscription_key = os.environ['BING_SEARCH_SUBSCRIPTION_KEY']
bing_search_url = os.environ['BING_SEARCH_ENDPOINT']

aoai_client = AzureOpenAI(azure_endpoint=apim_endpoint, 
                          api_key=apim_key,  
                          api_version=version_of_api
                        )


In [2]:
# query = "Why did the applicants fail in screening?"
# advanced_keywords = {
#     "+site": "https://kb.realpage.com/",
#     # "filetype": "pdf",
# }
# request_results_table = helpers.get_web_search_results(bing_subscription_key, bing_search_url, query, advanced_keywords)
# display(request_results_table)

In [3]:
def bing_web_search(api_key, query: str, output_format: str ='markdown', **kwargs): --> str:
    """
    Perform a search using the Bing Web Search v7.0 API with error handling and support for various query parameters.
    Outputs in either Markdown or HTML table format, including additional details about the search results.

    Args:
        api_key (str): The API key for accessing the Bing Web Search API.
        query (str): The user's search query term. Must not be empty.
        output_format (str): The format of the output, either 'markdown' or 'html'. Default is 'markdown'.
        **kwargs: Arbitrary keyword arguments representing additional query parameters supported by the API.

    Returns:
        str: A table with the search results in the specified format or an error message, including additional details.

    Example:
        >>> api_key = 'YOUR_BING_API_KEY'
        >>> query = 'Python programming'
        >>> print(bing_web_search(api_key, query, output_format='html', count=5, mkt='en-US'))
    """  
    base_url = "https://api.bing.microsoft.com/v7.0/search"  
    headers = {"Ocp-Apim-Subscription-Key": api_key}  
    params = {"q": query}  
    params.update(kwargs)  
  
    try:  
        response = requests.get(base_url, headers=headers, params=params)  
        response.raise_for_status()  
    except requests.exceptions.HTTPError as err:  
        return f"HTTP Error: {err}"  
    except requests.exceptions.RequestException as e:  
        return f"Error: {e}"  
  
    results = response.json()  
  
    if 'webPages' not in results:  
        return "No results found or the query was invalid."  
  
    if output_format == 'markdown':  
        table = "Name (URL) | Snippet | Last Updated | Type\n"  
        table += "--- | --- | --- | ---\n"  
    elif output_format == 'html':  
        table = "<table><tr><th>Name (URL)</th><th>Snippet</th><th>Last Updated</th><th>Type</th></tr>"  
  
    for item in results.get("webPages", {}).get("value", []):  
        name = html.escape(item.get("name", "N/A"))  
        url = item.get("url", "N/A")  
        snippet = item.get("snippet", "N/A").replace("\n", " ")  
        last_updated = item.get("dateLastCrawled", "N/A")[:10]  # Extract just the date  
        result_type = "Web page"  # In this context, all results are web pages  
          
        # Handle snippet highlighting  
        if output_format == 'html':  
            snippet = html.escape(snippet, quote=False)  
        else:  # For Markdown, convert highlight tags from HTML to Markdown if present  
            snippet = snippet.replace('<strong>', '**').replace('</strong>', '**')  
          
        # Format the name and URL as a clickable link  
        if output_format == 'html':  
            link = f"<a href='{url}'>{name}</a>"  
        else:  # Markdown  
            link = f"[{name}]({url})"  
          
        # Build the table row  
        if output_format == 'markdown':  
            table += f"{link} | {snippet} | {last_updated} | {result_type}\n"  
        elif output_format == 'html':  
            table += f"<tr><td>{link}</td><td>{snippet}</td><td>{last_updated}</td><td>{result_type}</td></tr>"  
  
    table += "</table>" if output_format == 'html' else ""  
  
    return table

In [5]:
output = bing_web_search(bing_subscription_key, "Why did the applicants fail in screening? site:kb.realpage.com", output_format='markdown', count=50, mkt='en-US')
display(Markdown(output))

Name (URL) | Snippet | Last Updated | Type
--- | --- | --- | ---
[Product Support Knowledge Base - RealPage](https://kb.realpage.com/articles/Solution/Screening-Failed-for-Applicants-with-Guarantors) | Why This Happens. The guarantor was screened separately causing the result to Fail. Applicant's income was not excluded causing it to fail. Income was set to not combine with other group applicant type. Before You Begin The user must have the rights to create and edit general screening settings to complete the steps. | 2024-01-31 | Web page
[Product Support Knowledge Base](https://kb.realpage.com/articles/Solution/New-Experience-Unable-to-Continue-Screening-Due-to-IDV-Conditional-Pending-OTP-Status) | If the applicant has entered an incorrect phone number, please have them update the number through their online application. Option 3: Applicant Continues to Fail or Reach Max Attempts You would need to follow your property's internal guidelines in validating an applicant's identity and process an IDV override. | 2024-02-13 | Web page
[Product Support Knowledge Base - kb.realpage.com](https://kb.realpage.com/articles/Solution/IDV-went-back-to-Pending-Status-from-Passed) | On-Site Screening. Resolution. Why This Happens Applicant Information was changed on the applicant. This will set the IDV Status back to Pending from Passed. Before You Begin. ... If the applicant receives a Fail. They will be routed to the Document Validation process in IDV. | 2024-01-26 | Web page
[Product Support Knowledge Base](https://kb.realpage.com/articles/Solution/LeasingDesk-Screening-What-is-Results-and-Decisions-Report) | Applicant Count: This is the number of applicants for the property and applicant type (if selected) during the date range. The last row of the column shows the sum of all applicants. ... Results – Fail – %: This is the percentage of applications that received a screening result of fail. In the last row of the column, all failing ... | 2024-01-28 | Web page
[LeasingDesk Screening: Verify the Passing or Failing Threshold of ...](https://kb.realpage.com/articles/Solution/Verify-the-Passing-Leasing-Desk-Score) | The user must have the right to create and edit general screening settings to complete these steps. Applicants pass with conditions when their score is between the Fail Threshold and Pass Threshold scores. Verify the Passing or Failing Threshold LeasingDesk Score | 2024-01-20 | Web page
[LeasingDesk Screening: Screening Error - Please Complete Screening Settings](https://kb.realpage.com/articles/Solution/Screening-Error-Please-Complete-Screening-Settings) | Support line: (800) 704-0154 > Option 8 > 1. Email: Support@RealPage.com. LeasingDesk Screening: Add a New Scoring Model to Property. | 2024-02-05 | Web page
[New OneSite: Make a Screening Decision - RealPage](https://kb.realpage.com/articles/Solution/New-Experience-Make-a-Screening-Decision) | To make a decision on a screen: 1. Go to the Navigation Menu > Accounts > Applicants or the appropriate waitlist 2. Locate the household and click on the household name 3. Go to the Lease Details tab 4. Go to Screening on the left 5. Next to the household member, click Actions > View Results 6. Review the screening results 7. | 2024-01-19 | Web page
[Product Support Knowledge Base](https://kb.realpage.com/articles/Solution/Re-screen-applicant-with-a-newly-added-Gurantor) | You can now re-screen the applicant and guarantor by clicking Screen Now and ensure you select both their names. Option 2: Screen the Applicant and Guarantor/Co-signer Separately Screening them separately is recommended for applicants who failed due to Unsatisfactory Credit so that the Guarantors result won't be affected. | 2024-02-14 | Web page
[Product Support Knowledge Base](https://kb.realpage.com/articles/Solution/LeasingDesk-Screening-What-does-the-IDV-Document-Validation-process-look-like-for-the-applicant) | An applicant who has reached max retakes can request a link for a new session. If the applicant does not successfully complete the Doc Validation process on the third attempt a Fail result will populate and the applicant would be subject to a manual review if appropriate. | 2024-02-14 | Web page
[Product Support Knowledge Base - RealPage](https://kb.realpage.com/articles/Solution/Screen-Now-box-is-greyed-out) | Why This Happens. The applicant's screening has already been run and is currently pending or completed. The system has timed out during the application or submission of the application. Duplicate name with old screening record was selected when entering the application; | 2024-01-27 | Web page
[Product Support Knowledge Base](https://kb.realpage.com/articles/Solution/Screening-Result-Failed-Conditional-due-to-Rent-to-Income-Ratio-Group-Applicant-Income) | Why This Happens. The wrong value was used to calculate the Rent to Income Ratio; The applicant's monthly income was not computed as a group versus their monthly rent. Before You Begin. Appropriate roles and rights are needed to perform the steps below. The user must have the rights to Create and edit general screening settings and View ... | 2024-02-13 | Web page
[Product Support Knowledge Base](https://kb.realpage.com/articles/Solution/Errors-in-Application-Error-in-Criminal-State-Thread-was-being-Aborted) | 1. Go to the Navigation Menu > Applicants 2. Locate the applicant and click View next to their name 3. Go to the Lease Details tab 4. Under the Screening history section, click View Result to open the Scorecard | 2024-02-11 | Web page
[Product Support Knowledge Base](https://kb.realpage.com/articles/Solution/Errors-in-application-Bad-XML-received-from-any-State) | Go to Navigation Menu > Applicants or Residents 2. Find and View the household in question 3. Go to Lease Details tab 4. Under Screening History, click Cancel Screen 5. Enter a Reason 6. Click Save. Step 2: Correct the Applicant's Information. 1. Go to Summary tab 2. Next to the name of the person you are screening, click Edit 3. Provide SSN of ... | 2024-02-13 | Web page
[LeasingDesk Screening: Screening Demographics Report](https://kb.realpage.com/articles/Solution/LeasingDesk-Screening-Screening-Demographics-Report) | 2. Screening Demographics by Site. This section provides charts for a percent comparison of the applicant count by applicant type. To expand applicant details at each property using the “+” sign to see the number of applicants that fall into specific category range for the items below: Age Group. Screening Scores. | 2024-02-14 | Web page
[Product Support Knowledge Base](https://kb.realpage.com/articles/Solution/Unable-to-re-screen-as-applicant-has-a-screening-status-Previously-Screened) | Applicant has a screening status 'Previously Screened' and you cannot proceed with their application. If that link is clicked under the Summary tab (AAAG page), it displays a message saying: "The contact was imported by conversion as "Previously Screened", as a result, details are not available". | 2024-01-26 | Web page
[Product Support Knowledge Base](https://kb.realpage.com/articles/Solution/Bureau-Error-201-INVALID-LAST-NAME) | You can now make changes to the application and screen the applicant as you normally would. For standalone screening, cancel the screen and create a new screen with the correct information. Additional Information . RealPage, Inc. 2201 Lakeside Blvd Richardson, TX 75082 (877) 325-7243. | 2024-01-22 | Web page
[Product Support Knowledge Base](https://kb.realpage.com/articles/Solution/Screening-Edit-Applicant-s-Income) | Edit the applicant income used in screening. 1. Go to Navigation Menu > Residents or Applicants 2. Locate the household and click View next to their name. 3. Click Lease Details tab 4. Under Screening history, click Not completed : The screening must not have a Final Decision. | 2024-02-04 | Web page
[LeasingDesk Screening: Scorecard Shows No Social Security Number in ...](https://kb.realpage.com/articles/Solution/Scorecard-Shows-No-Social-Security-Number-in-Applicant-Type) | 1. Go to the Navigation Menu > Applicants 2. Locate the Applicant and click View next to their name 3. Go to the Lease details tab . 4. Under Screening History, locate the household member and click Cancel Screen . 5. Enter Reason and click OK. You can now make changes to the application and rescreen the applicant as you normally would. | 2024-01-23 | Web page
[Product Support Knowledge Base](https://kb.realpage.com/articles/Solution/Re-screening-international-applicant-with-SSN) | Step 1: Cancel a Screening. 1. Go to the Navigation Menu > Applicants 2. Locate the Applicant and click View next to their name 3. Go to the Lease details tab. 4. Under Screening History, locate the household member and click Cancel Screen. 5. Enter Reason and click OK. You can now make changes to the application and rescreen the applicant as you normally would. | 2024-02-14 | Web page
[Product Support Knowledge Base](https://kb.realpage.com/articles/Solution/Screening-Error-The-format-of-the-identification-source-above-does-not-match-the-standard-designed-by-the-state) | Rerun the screen. Option 2: Screen Applicant With Another Type of ID : If this is not a license or non driver ID, you can screen the applicant with another type of ID. 1. Close out of the screen window. 2. Under Actions, click Screen now. 3. Select the applicant/s 4. Click Screen . 5. Enter the Applicant's correct Information; | 2024-02-06 | Web page
[Product Support Knowledge Base](https://kb.realpage.com/articles/Solution/Unable-to-Screen-Select-Applicants-as-the-Box-is-Grayed-Out-Screening-of-International-Address-is-Currently-Not-Supported) | You are trying to screen international applicants but the check box grayed out. You are also getting a pop up indicating the following: "Screening of international addresses is currently not supported." "The International, No Social Security Number and/or International Guarantor applicant type must be set up in order to screen this individual." | 2024-02-05 | Web page
[Product Support Knowledge Base](https://kb.realpage.com/articles/Instructional/Cancel-Out-Of-Network) | An Out of Network Screen, or OON, is a supplemental screening run in districts that do not allow automated or digital access to their records. They typically require a clerk to search paper records for information relating to your applicant manually. An Out-of-network search takes 1-4 business days to complete. | 2024-02-13 | Web page
[Product Support Knowledge Base](https://kb.realpage.com/articles/Solution/The-Apply-now-Screen-now-action-cannot-be-performed-to-a-prospect-whose-status-is-now-Leased) | "The 'Apply now'/'Screen now' action cannot be performed to a prospect whose status is now Leased". ... The prospect is already an applicant or a waitlisted applicant in the system meaning they are already assigned to a unit. Locate the Application in the Applicants or Find Person tab. 1. Go to either: a. Navigation menu > Applicants > Find ... | 2024-01-30 | Web page
